In [1]:
import pandas as pd

# dropping columns ans shuffling datasets 
bank_df = pd.read_csv("train.csv", index_col="id")
bank_df = bank_df.drop(['CustomerId', 'Surname'], axis=1)
bank_df = bank_df.sample(frac=1)
bank_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
id,,,,,,,,,,,
94184,669,France,Female,28.0,6,154712.58,1,1.0,1.0,87569.92,0
116599,645,Spain,Female,33.0,9,0.00,2,0.0,1.0,193008.31,0
102408,559,Spain,Male,43.0,7,0.00,1,1.0,0.0,2498.67,1
106181,597,Germany,Male,33.0,5,131694.26,1,0.0,1.0,82970.97,0
148906,683,France,Male,51.0,5,138341.51,1,1.0,0.0,160110.65,1


In [3]:

# apply min-max scaling and conversion categorical features to intergerss
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

cat_col = [1,2]
num_col = [0,3,4,5,6,7,8,9]

# Filling missing categorical values
cat_impute = SimpleImputer(strategy="most_frequent")
bank_df.iloc[:,cat_col] = cat_impute.fit_transform(bank_df.iloc[:,cat_col])

# Filling missing numerical values
num_impute = SimpleImputer(strategy="median")
bank_df.iloc[:,num_col] = num_impute.fit_transform(bank_df.iloc[:,num_col])


# Encode categorical features as an integer array.
cat_encode = OrdinalEncoder()
bank_df.iloc[:,cat_col] = cat_encode.fit_transform(bank_df.iloc[:,cat_col])


# Scaling numerical values.
scaler = MinMaxScaler()
bank_df.iloc[:,num_col] = scaler.fit_transform(bank_df.iloc[:,num_col])

bank_df.head()

C:\Users\SubZero\AppData\Local\Temp\ipykernel_24640\3794916897.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.638 0.59  0.418 ... 0.5   0.922 0.506]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  bank_df.iloc[:,num_col] = scaler.fit_transform(bank_df.iloc[:,num_col])
C:\Users\SubZero\AppData\Local\Temp\ipykernel_24640\3794916897.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.6 0.9 0.7 ... 0.8 0.4 0.2]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  bank_df.iloc[:,num_col] = scaler.fit_transform(bank_df.iloc[:,num_col])
C:\Users\SubZero\AppData\Local\Temp\ipykernel_24640\3794916897.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.         0.33333333 0.         ... 0.

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
id,,,,,,,,,,,
94184,0.638,0.0,0.0,0.135135,0.6,0.616635,0.000000,1.0,1.0,0.437834,0
116599,0.590,2.0,0.0,0.202703,0.9,0.000000,0.333333,0.0,1.0,0.965076,0
102408,0.418,2.0,1.0,0.337838,0.7,0.000000,0.000000,1.0,0.0,0.012437,1
106181,0.494,1.0,1.0,0.202703,0.5,0.524891,0.000000,0.0,1.0,0.414837,0
148906,0.666,0.0,1.0,0.445946,0.5,0.551385,0.000000,1.0,0.0,0.800572,1


In [4]:
# combined two pieplines using column transodfpomer and provided index 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Identify numerical and categorical columns
cat_col = [1,2]
num_col = [0,3,4,5,6,7,8,9]

# Transformers for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# Transformers for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

# Combine transformers into a ColumnTransformer
preproc_pipe = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_col),
        ('cat', categorical_transformer, cat_col)
    ],
    remainder="passthrough"
)

# Apply the preprocessing pipeline
bank_df = preproc_pipe.fit_transform(bank_df)
bank_df[0]

array([0.638     , 0.13513514, 0.6       , 0.61663514, 0.        ,
       1.        , 1.        , 0.43783351, 0.        , 0.        ,
       0.        ])